Notebook to construct $A(\Phi,B)$ for the irreducible root system of type "key" and the 3-dimensional 2-generated algebra $B$ with $(a,b)= c$

In [1]:
key = "A3" #this is the type of the root system.
c = 1 #this is the value (a,b).

In [2]:
#supporting functions
def CheckEigenvalues(A,a):
    output = true;
    for i in range(A.dimension()):
        b= A.basis()[i];
        output = output and (1/2*(a*b) +(a*(a*(a*b))) == 3/2*(a*(a*b)));
    return output;

def CheckZeroZero(A,a):
    output = true;
    for i in range(A.dimension()):
        for j in range(A.dimension()):
            b= A.basis()[i];
            c= A.basis()[j];
            b_new = a*(a*b) -3/2*(a*b) + 1/2*b;
            c_new = a*(a*c) -3/2*(a*c) + 1/2*c;
            output = output and ( a*(b_new*c_new) == 0*(b_new*c_new) );
    return output;


def CheckHalfHalf(A,a):
    output = true;
    for i in range(A.dimension()):
        for j in range(A.dimension()):
            b= A.basis()[i];
            c= A.basis()[j];
            b_new = a*(a*b) -(a*b) ;
            c_new = a*(a*c) -(a*c);
            output = output and  ( a*(a*(b_new*c_new)) == a*(b_new*c_new) );
    return output;


def CheckZeroHalf(A,a):
    output = true;
    for i in range(A.dimension()):
        for j in range(A.dimension()):
            b= A.basis()[i];
            c= A.basis()[j];
            b_new = a*(a*b) -(a*b) ;
            c_new = a*(a*c) -3/2*(a*c) + 1/2*c;
        output = output and ( a*(b_new*c_new) == 1/2*(b_new*c_new) );
    return output;

def CheckJordanfusionlaw(A,a):
    output = (a*a == a);
    output = output and CheckEigenvalues(A,a);
    output = output and  CheckHalfHalf(A,a);
    output = output and CheckZeroHalf(A,a);
    output = output and CheckZeroZero(A,a);
    return output;

def IsAxial(A):
    n=A.dimension();
    output = true;
    i =0;
    while i < n and output:
            output = output and CheckJordanfusionlaw(A,A.basis()[i]);
            i+=1;
    return output;

def IsLikelyJordan(A):
    output = true;
    n = A.dimension();
    for i in [1..10]:
        x = A.random_element();
        y = A.random_element();
        output = output and (x*y)*(x*x) == x*(y*(x*x)) ;
    return output;

In [3]:
#root system stuff
L = LieAlgebra(QQ, cartan_type=key)
R = RootSystem(key)
lattice =R.root_lattice()
space = R.root_space()
alpha = space.basis()
roots= space.roots()
pos_roots = space.positive_real_roots()
short_roots = space.short_roots()
long_roots = space.long_roots()
E = L.basis()
alphacheck = space.alphacheck()
chev_basis = space.roots()+[ alphacheck[i] for i in [1,2] ]
hvee = sum(space.highest_root().associated_coroot().coefficients())+1
n=len(alpha)
V=VectorSpace(QQ,3*len(pos_roots))
basis_vectors = V.basis()

In [4]:
#sanity check to make sure multiplication will be correctly defined.
lem = len(pos_roots)
for i in range(lem):
    for j in [i+1..lem-1]:
        assert not (pos_roots[i]-pos_roots[j] in pos_roots);


In [5]:
eps = 1-4*c
#assumes 0<=i<j<3
def mult2gen(index,i,j):
    if i==0 and j ==1:
        return 1/4*basis_vectors[index+1]-1/4*basis_vectors[index+2] + c*basis_vectors[index];
    if i== 0 and j ==2:
        return 1/4*basis_vectors[index+2]-1/4*basis_vectors[index+1] + c*basis_vectors[index];
    if i==1 and j ==2:
        return 1/4*(1+eps)*basis_vectors[index+2]+1/4*(eps^2-1)*basis_vectors[index] + (c+eps-eps*c - 1/4*(eps^2+eps))*basis_vectors[index+1];
    

In [6]:
#define multiplication between axes, and construct algebra
def mult(index1,index2):
    if index1>index2:
        return mult(index2,index1);
    
    if index1 == index2:
        return basis_vectors[index1];
    
    root1 = pos_roots[floor(index1/3)];
    x1 = Integer(mod(index1,3));
    root2 = pos_roots[floor(index2/3)];
    x2 = Integer(mod(index2,3));
    if root1 == root2:
        return mult2gen(3*floor(index1/3),x1,x2);
    if root1.scalar(root2.associated_coroot()) == 0:
        return V.zero_vector();
    
    elif root1+root2 in pos_roots:
        sum_index = pos_roots.index(root1+root2);
        if x1==0:
            return 1/4*basis_vectors[index1] + 1/4*basis_vectors[index2] -1/4*basis_vectors[3*sum_index+x1+x2];
        elif (x1==1 and x2 == 0):
            return 1/4*basis_vectors[index1] + 1/4*basis_vectors[index2] -1/4*basis_vectors[3*sum_index+1];
        elif (x1==1 and x2 == 1):
            return 1/4*basis_vectors[index1] + 1/4*basis_vectors[index2] -1/4*eps*basis_vectors[3*sum_index]+1/4*eps*basis_vectors[3*sum_index+1] - 1/4*basis_vectors[3*sum_index+2];
        elif (x1==1 and x2 == 2):
            return 1/4*basis_vectors[index1] + 1/4*basis_vectors[index2] -1/4*basis_vectors[3*sum_index];
        elif (x1==2 and x2 == 0):
            return 1/4*basis_vectors[index1] + 1/4*basis_vectors[index2] -1/4*basis_vectors[3*sum_index+2];
        elif (x1==2 and x2 == 1):
            return 1/4*basis_vectors[index1] + 1/4*basis_vectors[index2] -1/4*basis_vectors[3*sum_index];
        elif (x1==2 and x2 == 2):
            return 1/4*basis_vectors[index1] + 1/4*basis_vectors[index2] -1/4*eps*basis_vectors[3*sum_index]-1/4*basis_vectors[3*sum_index+1] + 1/4*eps*basis_vectors[3*sum_index+2];
    elif root2-root1 in pos_roots:
        sum_index = pos_roots.index(root2-root1);
        if x1==0:
            return 1/4*basis_vectors[index1] + 1/4*basis_vectors[index2] -1/4*basis_vectors[3*sum_index+x1+x2];
        elif (x1==1 and x2 == 0):
            return 1/4*basis_vectors[index1] + 1/4*basis_vectors[index2] -1/4*basis_vectors[3*sum_index+2];
        elif (x1==1 and x2 == 1):
            return 1/4*basis_vectors[index1] + 1/4*basis_vectors[index2]  -1/4*basis_vectors[3*sum_index];
        elif (x1==1 and x2 == 2):
            return 1/4*basis_vectors[index1] + 1/4*basis_vectors[index2]-1/4*eps*basis_vectors[3*sum_index]-1/4*basis_vectors[3*sum_index+1] +1/4*eps*basis_vectors[3*sum_index+2];
        elif (x1==2 and x2 == 0):
            return 1/4*basis_vectors[index1] + 1/4*basis_vectors[index2] -1/4*basis_vectors[3*sum_index+1];
        elif (x1==2 and x2 == 1):
            return 1/4*basis_vectors[index1] + 1/4*basis_vectors[index2] -1/4*eps*basis_vectors[3*sum_index]+1/4*eps*basis_vectors[3*sum_index+1] -1/4*basis_vectors[3*sum_index+2];
        elif (x1==2 and x2 == 2):
            return 1/4*basis_vectors[index1] + 1/4*basis_vectors[index2]-1/4*basis_vectors[3*sum_index];
                   
def multmat(index):
    return Matrix([ mult(index,i) for i in [0..len(basis_vectors)-1] ]);

structure_constants = [ multmat(i) for i in [0..len(basis_vectors)-1] ];
A = FiniteDimensionalAlgebra(QQ,structure_constants)

In [7]:
print(IsAxial(A)) #checkaxiality
print(IsLikelyJordan(A)) #check if A is Jordan, false means no, true means "most likely"

True
False
